<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>


## Table of Contents



<font size = 3>

1.  <a href="#item1">Scraping the Wikipedia page for the table of Canada</a>

2.  <a href="#item2">Creating the dataframe with the geographical coordinates</a>

3.  <a href="#item3">Exploring the neighborhoods in Toronto</a>
    
4.  <a href="#item4">Clustering the Neighborhoods</a>
    
5.  <a href="#item5">Examine Clusters</a>

    </font>


 Download all the dependencies that we will need.

In [1]:
!pip install beautifulsoup4 requests
!pip install lxml
!pip install geopy
!pip install folium
!pip install --user "scikit_learn==0.22.2.post1"
import requests # library to handle requests
import pandas as pd # library to process data as dataframes
import numpy as np # library for vectorized computation
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets import make_blobs
from bs4 import BeautifulSoup # library for retreiving the URL

print('Folium installed')
print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: Cryptograp

<a id='item1'></a>

## 1. Scraping the Wikipedia page for the table of Canada

In [2]:
# Retreiving the URL and creating a Beautiful soup object.
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(url,'lxml')

# Create a list.
table_contents=[]
table=soup.find('table')

# Create a dictionary.
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [3]:
df.shape

(103, 3)

----
The codes above is for the first part of the assignment Q1 in the week 3.

----

<a id='item2'></a>

## 2.Using the csv file to create the dataframe with the geographical coordinates.

In [4]:
# read the csv file into DataFrame.
df2 = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
df2.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
# Remove the space in 'Postal Code'.
df2.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df2.head()

# add other columns into dataframe.
toronto_df = pd.merge(df, df2, on='PostalCode', how='inner')
toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


----
The codes above is for the first part of the assignment Q2 in the week 3.

----

<a id='item3'></a>

## 3.Exploring the neighborhoods in Toronto.

In [6]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [7]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials and Version¶

In [8]:
CLIENT_ID = '-' # your Foursquare ID
CLIENT_SECRET = '-' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: -
CLIENT_SECRET:-


In [9]:
# explore the first neighborhood in our dataframe.¶
toronto_df.loc[0, 'Neighborhood']

'Parkwoods'

In [10]:
# Get the neighborhood's latitude and longitude values.
neighborhood_latitude = toronto_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [11]:
# get the top 20 venues that are in Parkwoods
LIMIT = 20 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60cf000d3434764b7244925a'},
 'response': {'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e6696b6d16433b9ffff47c3',
       'name': 'KFC',
       'location': {'lat': 43.75438666345904,
        'lng': -79.3330206627504,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.75438666345904,
          'lng': -79.3330206627504}],
        'distance': 298,
        'cc': 'CA',
        'country': 'Canada',
        'format

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,KFC,Fast Food Restaurant,43.754387,-79.333021
1,Brookbanks Park,Park,43.751976,-79.332140
2,TTC stop #8380,Bus Stop,43.752672,-79.326351
3,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [59]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [14]:
# create a function to repeat the same process to all the neighborhoods in Manhattan
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
northyork_df = toronto_df[toronto_df['Borough'] == 'North York'].reset_index(drop=True)
northyork_venues = getNearbyVenues(names=northyork_df['Neighborhood'],
                                   latitudes=northyork_df['Latitude'],
                                   longitudes=northyork_df['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills North
Glencairn
Don Mills South
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview East
York Mills, Silver Hills
Downsview West
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview Central
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


In [16]:
# check the size of the resulting dataframe
print(northyork_venues.shape)
northyork_venues.head()

(181, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
3,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [17]:
# check how many venues were returned for each neighborhood
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
Don Mills North,6,6,6,6,6,6
Don Mills South,18,18,18,18,18,18
Downsview Central,3,3,3,3,3,3
Downsview East,4,4,4,4,4,4
Downsview Northwest,6,6,6,6,6,6
Downsview West,6,6,6,6,6,6


In [18]:
#  find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(northyork_venues['Venue Category'].unique())))

There are 80 uniques categories.


Analyze each neighborhood.

In [19]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()
northyork_onehot.shape

(181, 81)

In [20]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Boutique,Bridal Shop,Burrito Place,Bus Stop,Business Service,Butcher,Café,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Fabric Shop,Fast Food Restaurant,Food & Drink Shop,Food Service,Food Truck,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Liquor Store,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Park,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoke Shop,Sporting Goods Shop,Supermarket,Sushi Restaurant,Thai Restaurant,Toy / Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.100000,0.000000,0.000000,0.00,0.000000,0.000000,0.05,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.100000,0.00,0.00,0.050000,0.05,0.000000,0.000000,0.05,0.000000,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.05,0.05,0.0,0.00,0.050000,0.000000,0.000000,0.00,0.000000,0.05,0.00,0.05,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.050000,0.050000,0.050000,0.00,0.00,0.0,0.00,0.00,0.00,0.000000,0.00,0.050000,0.050000,0.000000,0.000000,0.000000,0.05,0.00,0.00,0.000000,0.000000
1,Bayview Village,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.250000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.00,0.05,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.050000,0.050000,0.000000,0.000000,0.000000,0.000000,0.100000,0.05,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.05,0.050000,0.000000,0.000000,0.00,0.000000,0.00,0.05,0.00,0.050000,0.000000,0.050000,0.0,0.00,0.00,0.000000,0.000000,0.050000,0.050000,0.00,0.00,0.0,0.00,0.05,0.00,0.100000,0.00,0.100000,0.000000,0.000000,0.000000,0.000000,0.05,0.05,0.00,0.000000,0.000000
3,Don Mills North,0.000000,0.00,0.00,0.000000,0.000000,0.166667,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.166667,0.166667,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.166667,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.00,0.000000,0.166667,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.166667,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000
4,Don Mills South,0.000000,0.00,0.00,0.055556,0.055556,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.055556,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.055556,0.000000,0.000000,0.055556,0.055556,0.055556,0.00,0.00,0.000000,0.00,0.000000,0.055556,0.00,0.000000,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.00,0.055556,0.111111,0.000000,0.00,0.000000,0.00,0.00,0.00,0.055556,0.000000,0.000000,0.0,0.00,0.00,0.

In [21]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                venue  freq
0                Bank  0.10
1         Coffee Shop  0.10
2         Gas Station  0.05
3  Chinese Restaurant  0.05
4           Gift Shop  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1  Japanese Restaurant  0.25
2                 Café  0.25
3                 Bank  0.25
4             Pharmacy  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0          Restaurant  0.10
1         Coffee Shop  0.10
2      Sandwich Place  0.10
3  Italian Restaurant  0.05
4            Pharmacy  0.05


----Don Mills North----
                  venue  freq
0   Japanese Restaurant  0.17
1                   Gym  0.17
2  Caribbean Restaurant  0.17
3    Athletics & Sports  0.17
4                  Café  0.17


----Don Mills South----
                venue  freq
0                 Gym  0.11
1          Restaurant  0.11
2          Beer Store  0.06
3  Italian Restauran

In [81]:
# put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [82]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Convenience Store,Chinese Restaurant,Ice Cream Shop,Park,Pharmacy,Pizza Place,Bridal Shop,Grocery Store
1,Bayview Village,Chinese Restaurant,Café,Japanese Restaurant,Bank,Dessert Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Pharmacy,Pizza Place,Grocery Store,Greek Restaurant,Comfort Food Restaurant,Indian Restaurant,Pub
3,Don Mills North,Caribbean Restaurant,Dessert Shop,Café,Japanese Restaurant,Athletics & Sports,Gym,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
4,Don Mills South,Restaurant,Gym,Dim Sum Restaurant,Café,Clothing Store,Coffee Shop,Beer Store,Grocery Store,Italian Restaurant,Chinese Restaurant


<a id='item4'></a>

## 4.Clustering Neighborhoods¶

Run k-means to cluster the neighborhood into 5 clusters.

In [83]:
# set number of clusters
kclusters = 4

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 1, 2, 2, 2], dtype=int32)

In [84]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

northyork_merged = northyork_df

# merge northyork_grouped with northyork_df to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [85]:
# clean the dataframe
northyork_merged.dropna(inplace=True)
northyork_merged['Cluster Labels'] =northyork_merged['Cluster Labels'].astype(int)
northyork_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Park,Food & Drink Shop,Fast Food Restaurant,Bus Stop,Deli / Bodega,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Women's Store,Deli / Bodega,Chinese Restaurant,Clothing Store,Comfort Food Restaurant,Construction & Landscaping
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2,Clothing Store,Women's Store,Miscellaneous Shop,Boutique,Carpet Store,Coffee Shop,Vietnamese Restaurant,Furniture / Home Store,Accessories Store,Sandwich Place
3,M3B,North York,Don Mills North,43.745906,-79.352188,2,Caribbean Restaurant,Dessert Shop,Café,Japanese Restaurant,Athletics & Sports,Gym,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
4,M6B,North York,Glencairn,43.709577,-79.445073,2,Park,Pizza Place,Playground,Japanese Restaurant,Women's Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant
5,M3C,North York,Don Mills South,43.725900,-79.340923,2,Restaurant,Gym,Dim Sum Restaurant,Café,Clothing Store,Coffee Shop,Beer Store,Grocery Store,Italian Restaurant,Chinese Restaurant
6,M2H,North York,Hillcrest Village,43.803762,-79.363452,1,Fast Food Restaurant,Golf Course,Mediterranean Restaurant,Dog Run,Pool,Women's Store,Convenience Store,Chinese Restaurant,Clothing Store,Coffee Shop
7,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,2,Bank,Coffee Shop,Convenience Store,Chinese Restaurant,Ice Cream Shop,Park,Pharmacy,Pizza Place,Bridal Shop,Grocery Store
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,2,Coffee Shop,Bank,Clothing Store,Fast Food Restaurant,Bakery,Burrito Place,Japanese Restaurant,Restaurant,Salon / Barbershop,Shopping Mall
9,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,2,Caribbean Restaurant,Metro Station,Coffee Shop,Bar,Dessert Shop,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Deli / Bodega


Finally, let's visualize the resulting clusters

In [86]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighborhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## Examine Clusters

examine each cluster and determine the discriminating venue categories that distinguish each cluster.

In [87]:
# cluster 0
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Women's Store,Deli / Bodega,Chinese Restaurant,Clothing Store,Comfort Food Restaurant,Construction & Landscaping


In [88]:
# cluster 1
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Park,Food & Drink Shop,Fast Food Restaurant,Bus Stop,Deli / Bodega,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
6,North York,1,Fast Food Restaurant,Golf Course,Mediterranean Restaurant,Dog Run,Pool,Women's Store,Convenience Store,Chinese Restaurant,Clothing Store,Coffee Shop
11,North York,1,Park,Airport,Business Service,Construction & Landscaping,Dessert Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store
14,North York,1,Park,Basketball Court,Construction & Landscaping,Bakery,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Deli / Bodega
22,North York,1,Park,Convenience Store,Bar,Dessert Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Deli / Bodega


In [89]:
# cluster 2
northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,North York,2,Clothing Store,Women's Store,Miscellaneous Shop,Boutique,Carpet Store,Coffee Shop,Vietnamese Restaurant,Furniture / Home Store,Accessories Store,Sandwich Place
3,North York,2,Caribbean Restaurant,Dessert Shop,Café,Japanese Restaurant,Athletics & Sports,Gym,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
4,North York,2,Park,Pizza Place,Playground,Japanese Restaurant,Women's Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant
5,North York,2,Restaurant,Gym,Dim Sum Restaurant,Café,Clothing Store,Coffee Shop,Beer Store,Grocery Store,Italian Restaurant,Chinese Restaurant
7,North York,2,Bank,Coffee Shop,Convenience Store,Chinese Restaurant,Ice Cream Shop,Park,Pharmacy,Pizza Place,Bridal Shop,Grocery Store
8,North York,2,Coffee Shop,Bank,Clothing Store,Fast Food Restaurant,Bakery,Burrito Place,Japanese Restaurant,Restaurant,Salon / Barbershop,Shopping Mall
9,North York,2,Caribbean Restaurant,Metro Station,Coffee Shop,Bar,Dessert Shop,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Deli / Bodega
10,North York,2,Chinese Restaurant,Café,Japanese Restaurant,Bank,Dessert Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
13,North York,2,Grocery Store,Hotel,Park,Shopping Mall,Bank,Women's Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
15,North York,2,Furniture / Home Store,Pizza Place,Gym,Dessert Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store


In [90]:
# cluster 3
northyork_merged.loc[northyork_merged['Cluster Labels'] == 3, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,North York,3,Park,Food Truck,Carpet Store,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Deli / Bodega


Cluster0: It is the Victoria Village at the edge of the North York.<br>Cluster1: They are parks near the Large-scale venues like golf venue, airort and Basketball Court.<br>Cluster2: It is the downtown of NorthYork.<br>Cluster3: There is only single record in Cluster3 in Willowdale, Newtonbrook	
